<a href="https://colab.research.google.com/github/aswinsuresh-04/Regression_Analysis/blob/main/Stock_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
pip install statsmodels plotly


In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go
from tabulate import tabulate  # Import tabulate for table formatting

# Load the data
stock_data = pd.read_csv('/content/INTC.csv')

# Convert the date column to datetime format and set as index
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.set_index('Date', inplace=True)

# Prepare the data for regression models
X = stock_data[['Open', 'High', 'Low', 'Volume']]
y = stock_data['Close']

# Split the data into training and testing sets for regression models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
lin_model = LinearRegression()
lasso_model = Lasso(alpha=0.1)
ridge_model = Ridge(alpha=0.1)

# Fit Linear Regression model
lin_model.fit(X_train, y_train)
y_pred_lin = lin_model.predict(X_test)

# Fit Lasso Regression model
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)

# Fit Ridge Regression model
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

# Fit the ARIMA model
arima_model = ARIMA(stock_data['Close'], order=(5, 1, 0))
arima_model_fit = arima_model.fit()

# Make predictions with ARIMA for the same period
arima_forecast = arima_model_fit.forecast(steps=len(X_test))

# Calculate performance metrics for all models
metrics = {
    'Model': ['Linear Regression', 'Lasso Regression', 'Ridge Regression', 'ARIMA'],
    'Mean Squared Error (MSE)': [
        mean_squared_error(y_test, y_pred_lin),
        mean_squared_error(y_test, y_pred_lasso),
        mean_squared_error(y_test, y_pred_ridge),
        mean_squared_error(y_test, arima_forecast)
    ],
    'R² Score': [
        r2_score(y_test, y_pred_lin),
        r2_score(y_test, y_pred_lasso),
        r2_score(y_test, y_pred_ridge),
        r2_score(y_test, arima_forecast)
    ]
}

# Create a summary table comparing model performance
summary_table = pd.DataFrame(metrics)

# Use tabulate to display the summary table with better formatting
print("Summary of Model Performance:")
print(tabulate(summary_table, headers='keys', tablefmt='pretty', showindex=False))

# Create an interactive plot using Plotly for stock prices and predictions
fig = go.Figure()

# Add the actual closing prices
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Actual Prices', line=dict(color='blue')))

# Add predictions for each model
fig.add_trace(go.Scatter(x=X_test.index, y=y_pred_lin, mode='lines', name='Linear Regression Predictions', line=dict(color='green')))
fig.add_trace(go.Scatter(x=X_test.index, y=y_pred_lasso, mode='lines', name='Lasso Regression Predictions', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=X_test.index, y=y_pred_ridge, mode='lines', name='Ridge Regression Predictions', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=X_test.index, y=arima_forecast, mode='lines', name='ARIMA Predictions', line=dict(color='red')))

# Ups and downs logic for the stock prices
stock_data['Change'] = stock_data['Close'].diff()
stock_data['Direction'] = np.where(stock_data['Change'] > 0, 'Up', 'Down')

# Add trace for ups and downs
for i in range(1, len(stock_data)):
    if stock_data['Direction'].iloc[i] == 'Up':
        fig.add_trace(go.Scatter(
            x=[stock_data.index[i-1], stock_data.index[i]],
            y=[stock_data['Close'].iloc[i-1], stock_data['Close'].iloc[i]],
            mode='lines',
            line=dict(color='green', width=3),
            showlegend=False
        ))
    else:
        fig.add_trace(go.Scatter(
            x=[stock_data.index[i-1], stock_data.index[i]],
            y=[stock_data['Close'].iloc[i-1], stock_data['Close'].iloc[i]],
            mode='lines',
            line=dict(color='red', width=3),
            showlegend=False
        ))

# Update layout
fig.update_layout(title='Intel Stock Prices with Various Regression Predictions',
                  xaxis_title='Date',
                  yaxis_title='Price (USD)',
                  legend_title='Legend',
                  template='plotly_white')

# Show the plot
fig.show()


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.552e+02, tolerance: 2.630e+02

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning:

Ill-conditioned matrix (rcond=4.80267e-17): result may not be accurate.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWar

Summary of Model Performance:
+-------------------+--------------------------+--------------------+
|       Model       | Mean Squared Error (MSE) |      R² Score      |
+-------------------+--------------------------+--------------------+
| Linear Regression |   0.05299425262139017    |  0.99982340866226  |
| Lasso Regression  |   0.17477733169016552    | 0.9994175941487412 |
| Ridge Regression  |   0.05299371558177003    | 0.9998234104518229 |
|       ARIMA       |    1326.4455297309198    | -3.420079139672871 |
+-------------------+--------------------------+--------------------+
